In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m

# Datos

In [2]:
span = 250

atm_statuses =[{}, 
    {
        't': -10,
        'w': 0,
        's': 11.0, 
    },{
        't': 10,
        'w': 130,
        's': 11, 
    },{
        't': -5,
        'w': 50,
        's': 11.0, 
    },{
        't': 50,
        'w': 0,
        's': 11.0, 
    },{
        't': 15,
        'w': 0,
        's': 6.5, 
    }
]

conductor = {
    's_nominal':'185/30', #mm2
    's_efec': 213.6, #mm2
    'diameter': 19,#mm
    'weight':740.8, #daN/km
    'E':7700,#daN/mm2
    'alfa':.0000189 #1/°C    
}

def convert_kmph_mps(kmph):
    return kmph/3.6

def convert_mm_m(mm):
    return mm/1000

def convert_m_mm(m):
    return m*1000

# Tabla III

In [3]:
def getCFromTableIII(diameter):
    if not diameter or diameter < 0 or not isinstance(diameter, (int, float)):
        raise Exception('El Diametro debe ser positivo')
    
    if diameter<=12.5:
        return 1.2
    elif 12.5<diameter<15.8:
        return 1.1
    elif diameter>=15.8:
        return 1.0
    else:
        return 1.3   

try:
    getCFromTableIII(-1)
except Exception as e:
    print(e)
    

El Diametro debe ser positivo


# Cargas de Viento 
### Para vanos menores a 200m
$$W=0.75 C \left(\frac{V^2}{16}\right) d$$
### Para vanos mayores a 200m
$$W=0.75 C \left(\frac{V^2}{16}\right) d \left(80+0.6a\right)$$

In [4]:
def _W(C, wind_vel, conductor_diameter, span):
    if C and wind_vel and conductor_diameter and span <= 200:
        return .75*C*((wind_vel**2)/16)*conductor_diameter*span
    else:
        return .75*C*((wind_vel**2)/16)*conductor_diameter*(80+.6*span)
    

# Cargas Verticales
$$ g_v = \frac{p}{s} \left[\frac{1}{daN mm^2}\right] $$
p : Peso del cable por unidad de longitud, en $\frac{1}{daN.m}$ <br>
s : Sección transversal del cable, en mm2

# Cargas Horizontales

$$ w=\frac{W}{a} $$

$$ g_v = \frac{w}{s} \left[\frac{1}{daN mm^2}\right] $$

w : Carga del viento por unidad de longitud, en $\frac{1}{daN.m}$ <br>


In [5]:
try:
    g = []
    C = getCFromTableIII( conductor['diameter']) #obtener C desde la tabla con la funcion "getCFromTableIII"

    for status  in atm_statuses: #recorrer todos los estados atmosfericos
        if status: # evitar el estado con indice cero
            gp = conductor['weight'] * .001 / conductor['s_efec'] # carga vertical por u. de long gp = peso/seccion en daN/m

            W = _W( #calcular W con la funcion "calcWindLoad"
                C, # C obtenido previamente
                convert_kmph_mps( status['w']), # velocidad del viento para este estado, en m/s
                convert_mm_m( conductor['diameter']), # diametro del conductor en m
                span ) # vano de calculo en m

            w = W / span #carga del viento por vano w=W/a

            gv = w / conductor['s_efec'] # carga vertical por u. de long gv = w/s

            _g = m.sqrt( gp**2+gv**2) # g = raiz gv2 + gp2

            g.append( _g) 

            status['g'] = _g
except Exception as e:
    print('Error en la Tabla III',e)

# Ecuación de Cambio de Estado

$$  \sigma_j^3-A\sigma_j^2=B  $$


Siendo:

<br>
$ A = \sigma_i - \alpha E(t_j-t_i)-\frac{g_i^2a^2}{\sigma_i^2}\frac{E}{24} $
<br>
$ B = \frac{g_j^2a^2E}{24} $

In [6]:
class StatusChangesEcuation():
    _ec_params = {
                     'adm_strength' : None, 
                     'alfa' : None, 
                     'E' : None, 
                     'temp_j' : None, 
                     'temp_i' : None, 
                     'load_i' : None, 
                     'load_j' : None, 
                     'span' : None
                 }
    A = None
    B = None
    
    def __init__(self,_input):
        self._ec_params['strength_i'] = _input['status_i']['s'] 
        self._ec_params['strength_j'] = _input['status_j']['s'] 
        self._ec_params['alfa'] = _input['conductor']['alfa'] 
        self._ec_params['E'] = _input['conductor']['E'] 
        self._ec_params['temp_j'] = _input['status_j']['t'] 
        self._ec_params['temp_i'] = _input['status_i']['t'] 
        self._ec_params['load_i'] = _input['status_i']['g'] 
        self._ec_params['load_j'] = _input['status_j']['g'] 
        self._ec_params['span'] = _input['span']
        

    def _A(self, strength_i, alfa, E, temp_j, temp_i, load_i, span):
        return strength_i - alfa*E*( temp_j - temp_i ) - (load_i**2)*(span**2)*E/(24*strength_i**2)

    def _B(self, load_j, span, E):
        return (load_j**2)*(span**2)*E/24

    def calcStrength(self):

        self.A = self._A(self._ec_params['strength_i'], 
               self._ec_params['alfa'], 
               self._ec_params['E'], 
               self._ec_params['temp_j'], 
               self._ec_params['temp_i'], 
               self._ec_params['load_i'], 
               self._ec_params['span'])
        
        self.B = self._B(self._ec_params['load_j'], 
               self._ec_params['span'], 
               self._ec_params['E'])

        return  np.abs( 
                    np.polynomial.Polynomial([ -self.B ,0, -self.A ,1]).roots()
                ).max()
               

    def checkStrength(self):

        return np.greater( self._ec_params['strength_j'] , self.calcStrength() )



In [9]:

for n in range(1,6):
    base = atm_statuses[n]
    checklist = []
    tableIX = []
    for current_status in atm_statuses:
        if current_status:
            calc = StatusChangesEcuation({ 
                'status_i' : base, 
                'status_j' : current_status,
                'conductor' : conductor,
                'span': span
            })

            if calc.checkStrength():
                check = 'Si'
                checklist.append(1)
            else:
                check = 'No'
                checklist.append(0)

            if len(checklist)==5 and all(checklist):
                print('El Estado {} verifica'.format(n)) 
            
            tableIX.append([calc.A,calc.B,calc.calcStrength(),current_status['s'],check ])
                
pd.DataFrame(tableIX,columns=['A','B','Tension Calculada','Tension Admisible','Verifica'])

El Estado 5 verifica


,A,B,Tension Calculada,Tension Admisible,Verifica
0,4.429615,241.189808,8.103004,11.0,Si
1,1.519015,742.938014,9.592694,11.0,Si
2,3.701965,252.169557,7.822722,11.0,Si
3,-4.302185,241.189808,6.895673,11.0,Si
4,0.791365,241.189808,6.500000,6.5,Si
